In [1]:
# set up the python
import cosima_cookbook as cc
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import numpy as np
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs
import cmocean as cm
import cartopy.feature as cft
import cftime
import IPython.display
import sys, os, warnings

got 1998-mean SSH of MOM6

In [2]:
# load the MOM6 data
db='/g/data/x77/amh157/PanAnt/panant.db'
session=cc.database.create_session(db)
expts = ['panant-v2','panant-hycom1','panant-v3','panant-hycom1-v2','panant-01-zstar']

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37353,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://10.0.128.159:46093,Total threads: 4
Dashboard: /proxy/35369/status,Memory: 11.23 GiB
Nanny: tcp://127.0.0.1:38933,


In [4]:
# get the depth
depth = cc.querying.getvar('panant-v2','deptho',session,n=1)
land_mask = np.squeeze(depth.values)*0
land_mask[np.isnan(land_mask)] = 1
land_mask = np.where(land_mask==1,land_mask,np.nan)

In [5]:
# get 1-year SSH value of MOM6 
zos = cc.querying.getvar('panant-v2','zos', session,chunks={'xh':120,'yh':120})
zos = zos.isel(time=slice(2555,2920))

In [6]:
# check the time period
t0 = zos.time.item(0).strftime("%Y-%m-%d"),
t1 = zos.time.item(-1).strftime("%Y-%m-%d"),

print([t0,t1])

[('1998-01-01',), ('1998-12-31',)]


In [7]:
# calculate the annual mean of SSH (MOM6)
zos_mean = zos.mean('time').load()

And then ACCESS-OM2

In [8]:
session_2 = cc.database.create_session()

In [9]:
#SSH variable in ACCESS-OM2 models
variable = 'sea_level'

# dates to match the AVISO record
start_time = '1998-01-01'
end_time = '1998-12-31'

In [10]:
# 1 degree SSH
expt_1 = '1deg_jra55v13_iaf_spinup1_B1'  # 1-deg experiment
ssh1 = cc.querying.getvar(expt_1, variable, session, start_time=start_time, end_time=end_time)
ssh1 = ssh1.sel(time=slice(start_time, end_time)) # slice to the same time range as MOM6
ssh1_mean = ssh.mean('time').load()

VariableNotFoundError: No files were found containing 'sea_level' in the '1deg_jra55v13_iaf_spinup1_B1' experiment

In [11]:
# 0.25 degree mean SSH
expt_2 = '025deg_jra55v13_ryf8485_gmredi6'  # 0.25-deg experiment
ssh025 = cc.querying.getvar(expt_2, variable, session, start_time=start_time, end_time=end_time)
ssh025 = ssh025.sel(time=slice(start_time, end_time)) # slice to the same time range as MOM6
ssh025_mean = ssh025.mean('time').load()

VariableNotFoundError: No files were found containing 'sea_level' in the '025deg_jra55v13_ryf8485_gmredi6' experiment

In [ ]:
# 0.1 degree mean SSH
expt_3 = '01deg_jra55v13_iaf'  # 0.25-deg experiment
ssh01 = cc.querying.getvar(expt_3, variable, session, start_time=start_time, end_time=end_time)
ssh01 = ssh01.sel(time=slice(start_time, end_time)) # slice to the same time range as MOM6
ssh01_mean = ssh01.mean('time').load()

plot annual mean SSH

In [ ]:
# 1 deg 
projection=ccrs.SouthPolarStereo()
fig = plt.figure(figsize=(8, 8), dpi=150)
ax = plt.subplot(2, 2, 1, projection=projection)
ax.coastlines(resolution='50m')
ax.add_feature(cft.LAND)
ax.set_extent([-280, 80, -82, -60], crs=ccrs.PlateCarree())

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
depth.plot.contour(ax=ax,levels=[1000],linewidths=0.5,colors='grey',transform=ccrs.PlateCarree())
ssh1_mean.plot(vmin=-2,vmax=2,transform=ccrs.PlateCarree(),cmap=cm.cm.thermal,cbar_kwargs={'shrink': 0.4})

# 0.25 deg
ax = plt.subplot(2, 2, 2, projection=projection)
ax.coastlines(resolution='50m')
ax.add_feature(cft.LAND)
ax.set_extent([-280, 80, -82, -60], crs=ccrs.PlateCarree())

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
depth.plot.contour(ax=ax,levels=[1000],linewidths=0.5,colors='grey',transform=ccrs.PlateCarree())
ssh025_mean.plot(vmin=-2,vmax=2,transform=ccrs.PlateCarree(),cmap=cm.cm.thermal,cbar_kwargs={'shrink': 0.4})

# 0.1 deg
ax = plt.subplot(2, 2, 3, projection=projection)
ax.coastlines(resolution='50m')
ax.add_feature(cft.LAND)
ax.set_extent([-280, 80, -82, -60], crs=ccrs.PlateCarree())

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
depth.plot.contour(ax=ax,levels=[1000],linewidths=0.5,colors='grey',transform=ccrs.PlateCarree())
ssh01_mean.plot(vmin=-2,vmax=2,transform=ccrs.PlateCarree(),cmap=cm.cm.thermal,cbar_kwargs={'shrink': 0.4})

#0.05 MOOM6
ax = plt.subplot(2, 2, 1, projection=projection)
ax.coastlines(resolution='50m')
ax.add_feature(cft.LAND)
ax.set_extent([-280, 80, -82, -60], crs=ccrs.PlateCarree())

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax.set_boundary(circle, transform=ax.transAxes)
depth.plot.contour(ax=ax,levels=[1000],linewidths=0.5,colors='grey',transform=ccrs.PlateCarree())
zos_mean.plot(vmin=-2,vmax=2,transform=ccrs.PlateCarree(),cmap=cm.cm.thermal,cbar_kwargs={'shrink': 0.4})